In [11]:
from octis.models.ETM_model.etm import ETM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real


In [12]:
dataset = Dataset()  
dataset.fetch_dataset("20NewsGroup")
#dataset.load_custom_dataset_from_folder("data/processed") # Our custom preprocessed dataset


In [22]:
model = ETM(num_topics = 10, vocab_size=3000, t_hidden_size=800, theta_act = 'relu', embeddings = None, train_embeddings = True, enc_drop = 0.0, rho_size= 5, emb_size= 10)

In [24]:
output = model.train_model(dataset)

AttributeError: 'ETM' object has no attribute 'train_model'